# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
import pickle

In [2]:
import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /home/keigito/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/keigito/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('Messages_Categories', engine)


In [4]:
df.shape

(26185, 40)

In [5]:
X = df.loc[:, "message"]
Y = df.iloc[:, 4:40]

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for token in tokens:
        clean_token = lemmatizer.lemmatize(token).lower().strip()
        clean_tokens.append(clean_token)
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

pipeline = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
#     (), # Feature engineering (word2vec/GloVe)
    ("clf", MultiOutputClassifier(RandomForestClassifier(n_estimators=100), n_jobs=4))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=4))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_pred = pipeline.predict(X_test)

/home/keigito/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [10]:
y_actu = y_test.values

In [11]:
results_dict = {}
for i in range(1, 37):
    predicted = "pred_" + str(i)
    actual = "actu_" + str(i)
    pred_values = []
    actu_values = []
    for ii in range(len(y_pred)):

        pred_values.append(int(y_pred[ii][i-1]))
        actu_values.append(int(y_actu[ii][i-1]))

    results_dict[predicted] = pred_values
    results_dict[actual] = actu_values
    

In [12]:
target_names = list(df.columns)
drop_names = ["id", "message", "original", "genre"]
for name in drop_names:
    target_names.remove(name)

In [13]:
labels = []
for name in target_names:
    labels.append(name)

In [14]:
for i in range(1, 37):
    pred = results_dict['pred_' + str(i)]
    actu = results_dict['actu_' + str(i)]
    
    print("\n### " + target_names[i-1] + " ###\n")
    print(classification_report(pred, actu))
    
    
    


### related ###

              precision    recall  f1-score   support

           0       0.29      0.75      0.42       574
           1       0.97      0.81      0.89      5964
           2       0.12      0.67      0.20         9

   micro avg       0.81      0.81      0.81      6547
   macro avg       0.46      0.74      0.50      6547
weighted avg       0.91      0.81      0.84      6547


### request ###

              precision    recall  f1-score   support

           0       0.99      0.89      0.94      5982
           1       0.44      0.90      0.59       565

   micro avg       0.89      0.89      0.89      6547
   macro avg       0.72      0.90      0.77      6547
weighted avg       0.94      0.89      0.91      6547


### offer ###

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6547
           1       0.00      0.00      0.00         0

   micro avg       1.00      1.00      1.00      6547
   macro avg       0.5

/home/keigito/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
parameters = {
#    'vectorizer__ngram_range': ((1, 1), (1, 2)),
#    'vectorizer__max_df': (0.5, 1.0),
    'vectorizer__max_features': (5000, 10000), # 397
    'tfidf__use_idf': (True, False), # 489 sec
#    'clf__estimator__n_estimators': [100, 200], # 
    'clf__estimator__min_samples_split': [3, 4] # 444 sec 
    }



cv = GridSearchCV(pipeline, param_grid=parameters, cv=5)

In [20]:
import time
t1 = time.time()
cv_results = cv.fit(X_train, y_train)
print("The optimization took {} Min.".format((time.time() - t1)/60.0))

/home/keigito/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/home/keigito/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/home/keigito/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/home/keigito/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/ex

The optimization took 39.82915318409602 Min.


There seems to be a memory leak issue with joblib.Parallel. I could run the optimization serially, but it would take too long, so I'll proceed with the best model I get from this optimization.

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
best_model = cv_results.best_estimator_

In [22]:
best_model

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=4))])

In [23]:
y_pred = best_model.predict(X_test)

In [24]:
y_actu = y_test.values

In [25]:
results_dict = {}
for i in range(1, 37):
    predicted = "pred_" + str(i)
    actual = "actu_" + str(i)
    pred_values = []
    actu_values = []
    for ii in range(len(y_pred)):

        pred_values.append(int(y_pred[ii][i-1]))
        actu_values.append(int(y_actu[ii][i-1]))

    results_dict[predicted] = pred_values
    results_dict[actual] = actu_values
    

In [26]:
target_names = list(df.columns)
drop_names = ["id", "message", "original", "genre"]
for name in drop_names:
    target_names.remove(name)

In [28]:
for i in range(1, 37):
    pred = results_dict['pred_' + str(i)]
    actu = results_dict['actu_' + str(i)]
    
    print("\n### " + target_names[i-1] + " ###\n")
    print(classification_report(pred, actu))
    
    
    


### related ###

              precision    recall  f1-score   support

           0       0.31      0.74      0.43       619
           1       0.97      0.82      0.89      5901
           2       0.22      0.41      0.28        27

   micro avg       0.81      0.81      0.81      6547
   macro avg       0.50      0.65      0.53      6547
weighted avg       0.90      0.81      0.84      6547


### request ###

              precision    recall  f1-score   support

           0       0.99      0.90      0.94      5932
           1       0.48      0.90      0.63       615

   micro avg       0.90      0.90      0.90      6547
   macro avg       0.73      0.90      0.78      6547
weighted avg       0.94      0.90      0.91      6547


### offer ###

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6547
           1       0.00      0.00      0.00         0

   micro avg       1.00      1.00      1.00      6547
   macro avg       0.5

              precision    recall  f1-score   support

           0       1.00      0.95      0.97      6515
           1       0.05      0.59      0.10        32

   micro avg       0.95      0.95      0.95      6547
   macro avg       0.53      0.77      0.54      6547
weighted avg       0.99      0.95      0.97      6547


### direct_report ###

              precision    recall  f1-score   support

           0       0.99      0.86      0.92      5944
           1       0.40      0.88      0.54       603

   micro avg       0.86      0.86      0.86      6547
   macro avg       0.69      0.87      0.73      6547
weighted avg       0.93      0.86      0.89      6547



/home/keigito/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [34]:
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
#     (), # Feature engineering (word2vec/GloVe)
    ("clf", MultiOutputClassifier(KNeighborsClassifier(), n_jobs=4))
])

In [35]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ..._params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'),
           n_jobs=4))])

In [37]:
y_pred = pipeline.predict(X_test)

In [38]:
y_actu = y_test.values

In [39]:
results_dict = {}
for i in range(1, 37):
    predicted = "pred_" + str(i)
    actual = "actu_" + str(i)
    pred_values = []
    actu_values = []
    for ii in range(len(y_pred)):

        pred_values.append(int(y_pred[ii][i-1]))
        actu_values.append(int(y_actu[ii][i-1]))

    results_dict[predicted] = pred_values
    results_dict[actual] = actu_values
    

In [40]:
target_names = list(df.columns)
drop_names = ["id", "message", "original", "genre"]
for name in drop_names:
    target_names.remove(name)

In [42]:
for i in range(1, 37):
    pred = results_dict['pred_' + str(i)]
    actu = results_dict['actu_' + str(i)]
    
    print("\n### " + target_names[i-1] + " ###\n")
    print(classification_report(pred, actu))
    
    
    


### related ###

              precision    recall  f1-score   support

           0       0.43      0.57      0.49      1183
           1       0.90      0.83      0.86      5353
           2       0.23      0.82      0.36        11

   micro avg       0.78      0.78      0.78      6547
   macro avg       0.52      0.74      0.57      6547
weighted avg       0.81      0.78      0.79      6547


### request ###

              precision    recall  f1-score   support

           0       0.98      0.88      0.93      5993
           1       0.39      0.78      0.52       554

   micro avg       0.88      0.88      0.88      6547
   macro avg       0.68      0.83      0.72      6547
weighted avg       0.93      0.88      0.89      6547


### offer ###

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6547
           1       0.00      0.00      0.00         0

   micro avg       1.00      1.00      1.00      6547
   macro avg       0.5

              precision    recall  f1-score   support

           0       0.98      0.85      0.91      6046
           1       0.30      0.77      0.43       501

   micro avg       0.84      0.84      0.84      6547
   macro avg       0.64      0.81      0.67      6547
weighted avg       0.93      0.84      0.87      6547



/home/keigito/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [45]:
pickle.dump(best_model, open("final_model.pickle", "wb"))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.